<a href="https://colab.research.google.com/github/brunorosilva/ai_especialization_usp/blob/master/IAD-004-Aprendizagem_de_Maquina_1/ExerciciosRegressaoLog%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de clientes com regressão logística
Uma popular rede de varejo de móveis e eletrodomésticos está muito preocupada com o número de clientes que atrasam o pagamento de suas prestações. Eles precisam entender o rpefil de quem está atrasando e vocês são os analistas desta empresa.

Para isso disponibilizaram um conjunto de dados. Cada caso corresponde a um cliente separado e registra várias informações demográficas e pessoais. Vocês devem analisar todos os dados relevantes do cliente e desenvolver programa para minimizar os riscos da empresa.

O conjunto de dados inclui informações sobre:

- A última coluna mostra se os clientes estão ou não atrasados na mensalidade - (0: não atraso; 1: atraso de mais de dois meses).
- As demais colunas são informações pessoais e demográficas sobre clientes - Idade (em anos), Sexo (0:M, 1:F), Estado civil (0:casado, 1:solteiro ou divorciado), Endereço (por zona), Renda (média de rendimentos), Posses (balanço de posses registradas), Fidelidade (quantas vezes já comprou a prazo na empresa), Empregado (anos que se encontra empregado na mesma empresa), Internet em casa (0: não, 1: sim).

Vocês devem construir um modelo de regressão logística. Seu projeto deve estar dividido em três partes: 
1. Pré processamento dos dados e seleção;
2. Treinamento e teste;
3. Avaliação.

##Pré processamento dos dados e seleção

Nessa etapa, verifique os dados, o número de linhas e colunas, separe (treinamento-teste) e normalize. Sua análise dá algum insight sobre a importância de cada dado? São todos relevantes? Existe correlação?

## Treinamento e teste

Vocês podem usar o modelo __LogisticRegression__ do pacote Scikit-learn. Essa função implementa a regressão logística e pode usar diferentes otimizadores numéricos para encontrar parâmetros, incluindo os _solvers_ `newton-cg`, `lbfgs`, `liblinear`, `sag` e `saga`. Você pode encontrar informações detalhadas sobre os prós e contras desses otimizadores se pesquisar na Internet.

A versão de Regressão Logística no Scikit-learn, suporta a regularização. A regularização é uma técnica usada para resolver o problema de overfitting em modelos de aprendizado de máquina.
O parâmetro $C$ indica o *inverso da força de regularização*, i é, valores menores especificam uma regularização mais forte.

Lembre-se que `skitlearn` exige que o rótulo seja inteiro.

## Avaliação

Com base na matriz de confusão, podemos calcular a __precisão__ e o __recall__ de cada rótulo:


*  __Precisão__ é a proporção de positivos classificados corretamente, ou seja, dos classificados como positivo, quantos são realmente positivos. É definido por: 
$$Precision = \frac{TP}{TP + FP}$$

*  __Recall__ é a proporção de positivos identificados corretamente, ou seja, o quão bom o modelo é para detectar positivos. É definido por: $$Recall =  \frac{TP}{TP + FN}$$

Maximizar a *precisão* minimizará o número de falsos positivos, enquanto maximizar o *recall* o número de falsos negativos.

Às vezes, queremos excelentes previsões da classe positiva. Queremos alta precisão e alto recall. Nesse caso, o F1 Score pode ser uma medida melhor para usar na busca de um equilíbrio entre *Precisão* e *Recall*.


* __F1 Score:__
Pode-se calcular o F1 Score para cada classe com base na precisão e no recall dessa classe. O F1 Score é a média ponderada (use da precisão e do recall, onde o F1 Score alcança seu melhor valor em 1 (precisão e recall perfeitas) e pior em 0. Depois que a precisão e a recuperação são calculadas para um problema de classificação binária ou multiclasse, as duas pontuações podem ser combinadas no cálculo da F-Measure.
$$F1 = 2 {Precision * Recall \over Precision + Recall}$$

Na biblioteca `sklearn.metrics` vocês podem obter diretamente a matriz de confusão usando `confusion_matrix` e as métricas citadas acima com `classification_report`.


E, finalmente, poderão responder:

Qual a média do F1 Score das duas classes?

In [38]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

import itertools
%matplotlib inline 
import matplotlib.pyplot as plt
import io
import urllib.request as request

## Lendo os dados

In [2]:
perfil_df = pd.read_csv(io.TextIOWrapper(request.urlopen("https://www.lsc.poli.usp.br/PerfilCliente.csv")),delimiter=';')
perfil_df.head()

,Idade,Sexo,Estado Civil,Endereco,Renda,Posses,Fidelidade,Empregado,Internet,Mensalidade
0,33,0,1,7,136,11,5,5,1,1
1,33,0,0,12,33,33,2,0,0,1
2,30,0,0,9,30,23,1,2,0,0
3,35,1,1,5,76,38,2,10,1,0
4,35,0,1,14,80,7,2,15,0,0


In [60]:
from sklearn.utils import resample
df_majority = perfil_df[perfil_df.Mensalidade==0]
df_minority = perfil_df[perfil_df.Mensalidade==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True, 
                                 n_samples=len(df_majority), 
                                 random_state=123)  

df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
df_upsampled.Mensalidade.value_counts()

1    142
0    142
Name: Mensalidade, dtype: int64

In [65]:
dados = df_upsampled.drop(['Mensalidade'], axis=1)
rotulos = df_upsampled['Mensalidade']
x_train, x_test, y_train, y_test = train_test_split(dados, rotulos, test_size=.2)

array([[19,  5],
       [ 8, 25]])

In [66]:
model = LogisticRegression()
pred = model.fit(x_train, y_train).predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.70      0.79      0.75        24
           1       0.83      0.76      0.79        33

    accuracy                           0.77        57
   macro avg       0.77      0.77      0.77        57
weighted avg       0.78      0.77      0.77        57



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [68]:
confusion_matrix(y_test, pred)

array([[19,  5],
       [ 8, 25]])